### Bottleneck Feature generation for cifar10 and traffic dataset using Vgg, Resnet and Inception

the code utilizes pre-trained CNN models to generate bottleneck features for a given dataset and saves them along with the labels. These bottleneck features can be used as input to train a different model or perform other tasks such as transfer learning.


In [36]:
# Checking Tensorflow version
import tensorflow as tf
tf.version.VERSION


'1.15.8'

#### Exploring cifar10 dataset

In [37]:
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

(X_train, y_train), (_, _) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [38]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(40000, 32, 32, 3)
(40000, 1)
(10000, 32, 32, 3)
(10000, 1)


#### Checking GPU Access

Here we have installed tensorflow-directml to access our AMD radeon graphics

TensorFlow with DirectML enables training and inference of complex machine learning models on a wide range of DirectX 12-compatible hardware

In [39]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:DML:0', device_type='DML')]

In [28]:
tf.test.is_gpu_available()

True

#### Importing Necessary Modules

In [29]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Input, AveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.datasets import cifar10
import pickle
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import keras.backend as K

#### Defining the dataset to train, network to choose and set batch_size and height, width and channel of input 

In [30]:

# Make bottleneck features for either cifar10 or traffic dataset. 
dataset = 'cifar10'

# The model to bottleneck, ex. 'vgg', 'inception', or 'resnet
network = 'vgg'

# The batch size for the generator
batch_size = 128

# Setting Height, Width and Channel of input 
h, w, ch = 224, 224, 3
if network == 'inception':
    h, w, ch = 299, 299, 3
    from keras.applications.inception_v3 import preprocess_input

img_placeholder = tf.placeholder("uint8", (None, 32, 32, 3))
resize_op = tf.image.resize_images(img_placeholder, (h, w), method=0)

#### Defining our generator Function to yield our batches of our Sample and batches of our labels

generator takes a session, data, labels, and batch size as input and generates batches of preprocessed data and corresponding labels using TensorFlow operations. The generator is used to feed the data into the model during the prediction phase.



In [31]:

def gen(session, data, labels, batch_size):
    def _f():
        start = 0
        end = start + batch_size
        n = data.shape[0]
 
        while True:
            X_batch = session.run(resize_op, {img_placeholder: data[start:end]})
            X_batch = preprocess_input(X_batch)
            y_batch = labels[start:end]
            start += batch_size
            end += batch_size
            if start >= n:
                start = 0
                end = batch_size

            print(start, end)
            yield (X_batch, y_batch)

    return _f

#### Creating model

creates the selected CNN model (ResNet50, VGG16, or InceptionV3) and returns the model instance.



In [40]:
def create_model():
    input_tensor = Input(shape=(h, w, ch))
    if network == 'vgg':
        model = VGG16(input_tensor=input_tensor, include_top=False)
        x = model.output
        x = AveragePooling2D((7, 7))(x)
        model = Model(model.input, x)
    elif network == 'inception':
        model = InceptionV3(input_tensor=input_tensor, include_top=False)
        x = model.output
        x = AveragePooling2D((8, 8), strides=(8, 8))(x)
        model = Model(model.input, x)
    else:
        model = ResNet50(input_tensor=input_tensor, include_top=False)
    return model

#### Loading the dataset selected either cifar10 or traffic signs and generating bottleneck features

loading the dataset (either CIFAR10 or a traffic signs) and splits it into training and validation sets

defining the output file names for saving the bottleneck features.

creating a TensorFlow session and sets it as the default session.

For the training dataset and validation dataset, the model.predict_generator method is called, passing the generator function and the number of samples in the training dataset. The resulting bottleneck features are saved along with the corresponding labels in a pickle file.

Steps i have taken 313 for training and 79 for validation using Steps = len(X_train)/batch_size in order to cover whole dataset.


In [33]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

if dataset == 'cifar10':
    (X_train, y_train), (_, _) = cifar10.load_data()
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
else:
    with open('data/train.p', mode='rb') as f:
        train = pickle.load(f)
    X_train, X_val, y_train, y_val = train_test_split(train['features'], train['labels'], test_size=0.33, random_state=0)

train_output_file = "{}_{}_{}.p".format(network, dataset, 'bottleneck_features_train')
validation_output_file = "{}_{}_{}.p".format(network, dataset, 'bottleneck_features_validation')

print("Resizing to", (w, h, ch))
print("Saving to ...")
print(train_output_file)
print(validation_output_file)

with tf.Session() as sess:
    K.set_session(sess)
    K.set_learning_phase(1)

    model = create_model()

    # The model.predict_generator function returns an array of predicted probabilities,
    # where each row corresponds to 
    # a sample in the test set, and each column represents the probability 
    # of the sample belonging to a specific class.
    print('Bottleneck training')
    train_gen = gen(sess, X_train, y_train, batch_size)
    # predict_generator Generates predictions for the input samples from a data generator
    bottleneck_features_train = model.predict_generator(train_gen(), steps=313)
    data = {'features': bottleneck_features_train, 'labels': y_train}
    pickle.dump(data, open(train_output_file, 'wb'))

    print('Bottleneck validation')
    val_gen = gen(sess, X_val, y_val, batch_size)
    bottleneck_features_validation = model.predict_generator(val_gen(), steps=79)
    data = {'features': bottleneck_features_validation, 'labels': y_val}
    pickle.dump(data, open(validation_output_file, 'wb'))

Resizing to (224, 224, 3)
Saving to ...
vgg_cifar10_bottleneck_features_train.p
vgg_cifar10_bottleneck_features_validation.p
Bottleneck training
128 256
256 384
384 512
512 640
640 768
768 896
896 1024
1024 1152
1152 1280
1280 1408
1408 1536
1536 1664
1664 1792
1792 1920
1920 2048
2048 2176
2176 2304
2304 2432
2432 2560
2560 2688
2688 2816
2816 2944
2944 3072
3072 3200
3200 3328
3328 3456
3456 3584
3584 3712
3712 3840
3840 3968
3968 4096
4096 4224
4224 4352
4352 4480
4480 4608
4608 4736
4736 4864
4864 4992
4992 5120
5120 5248
5248 5376
5376 5504
5504 5632
5632 5760
5760 5888
5888 6016
6016 6144
6144 6272
6272 6400
6400 6528
6528 6656
6656 6784
6784 6912
6912 7040
7040 7168
7168 7296
7296 7424
7424 7552
7552 7680
7680 7808
7808 7936
7936 8064
8064 8192
8192 8320
8320 8448
8448 8576
8576 8704
8704 8832
8832 8960
8960 9088
9088 9216
9216 9344
9344 9472
9472 9600
9600 9728
9728 9856
9856 9984
9984 10112
10112 10240
10240 10368
10368 10496
10496 10624
10624 10752
10752 10880
10880 11008
110

#### Reading the pickle file generated and exploring the features and labels

In [34]:
import pickle
import collections, numpy as np

training_file = "vgg_cifar10_bottleneck_features_train.p"
validation_file = "vgg_cifar10_bottleneck_features_validation.p"

with open(training_file, 'rb') as f:
    train_data = pickle.load(f)
with open(validation_file, 'rb') as f:
    validation_data = pickle.load(f)

In [35]:
print("Training Bottleneck features shape :-",train_data['features'].shape)
print("Training Bottleneck labels shape :-",train_data['labels'].shape)
print("Validation Bottleneck features shape :-",validation_data['features'].shape)
print("Validation Bottleneck labels shape :-",validation_data['labels'].shape)
print("Unique Classes :-", np.unique(train_data['labels']))
labels = train_data['labels'].reshape(train_data['labels'].shape[0])
counter = collections.Counter(labels)
print("Number of Examples in each Class :-", counter)

Training Bottleneck features shape :- (40000, 1, 1, 512)
Training Bottleneck labels shape :- (40000, 1)
Validation Bottleneck features shape :- (10000, 1, 1, 512)
Validation Bottleneck labels shape :- (10000, 1)
Unique Classes :- [0 1 2 3 4 5 6 7 8 9]
Number of Examples in each Class :- Counter({6: 4047, 4: 4033, 1: 4014, 8: 4011, 7: 4001, 0: 3996, 5: 3984, 2: 3984, 3: 3970, 9: 3960})
